In [8]:
import tensorflow as tf
import librosa
import numpy as np
import IPython.display as ipd
import os
import pandas as pd

In [9]:
X_0 = []
paths = librosa.util.find_files(r'C:\Users\misai\Программирование\Проекты\PhraseRecognition\orig\Без шума\Старт')
for i in paths:
    x, sr = librosa.load(i)
    mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=512, n_mfcc=20)
    mfcc = np.mean(mfcc, axis=1)
    X_0.append(mfcc)
np.shape(X_0)

(71, 20)

In [52]:
y_0 = np.zeros(np.shape(X_0)[0])

In [11]:
X_1 = []
voices_path = r'C:\Users\misai\Программирование\Проекты\PhraseRecognition\audiobooks\data\ru_open_stt\private_buriy_audiobooks_2\0'
for path in os.listdir(voices_path):
    paths = librosa.util.find_files(voices_path + '\\' + path)
    for file in paths:
        x, sr = librosa.load(file)
        mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=512, n_mfcc=20)
        mfcc = np.mean(mfcc, axis=1)
        X_1.append(mfcc)
np.shape(X_1)

(3359, 20)

In [53]:
y_1 = np.ones(np.shape(X_1)[0])

In [14]:
save = np.copy(X_1)

In [21]:
columns = list('mfcc' + str(i) for i in range(1, 21))

In [26]:
DF_0 = pd.DataFrame(X_0, columns=list('mfcc' + str(i) for i in range(1, 21)))

In [29]:
DF_1 = pd.DataFrame(X_1, columns=list('mfcc' + str(i) for i in range(1, 21)))

In [44]:
save_1 = pd.read_csv(r'C:\Users\misai\Программирование\Проекты\PhraseRecognition\DF_1.csv', index_col=0)

In [51]:
X = DF_0.append(DF_1, ignore_index=True)
X

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,-340.768799,75.068962,-2.449855,21.445436,-6.660470,1.396364,-23.702692,15.587738,-5.263852,8.058776,-5.062593,5.981391,-10.102416,2.169253,2.053474,-3.483684,0.299574,2.409102,-3.164918,4.179435
1,-319.350159,110.979500,-9.221969,24.480909,0.433996,-3.178642,-12.134595,8.180908,-13.884797,1.397449,-1.784678,3.135814,-4.784268,1.858990,-6.371924,-0.053081,-7.679573,-1.390087,-8.511072,-3.212606
2,-277.733917,86.339371,9.546513,27.260670,-1.580854,2.294679,-21.063057,-6.768252,-16.339190,3.448966,-0.185480,3.037778,-7.492875,-4.326860,-3.938322,-0.061976,-8.204382,-0.990154,-7.994271,1.662887
3,-287.441437,101.411537,9.845778,18.111305,0.953473,4.148739,-21.461174,3.548748,-11.676284,1.810830,-8.262000,-2.074018,-8.954288,0.328092,-4.562632,1.470902,-8.286371,2.301535,-9.539157,-0.675301
4,-292.986938,82.176956,5.167121,21.194124,0.813182,-5.446115,-21.213644,1.445883,-20.313686,4.439660,-1.148367,-7.161251,-7.837586,-1.914691,-7.601760,4.359953,-6.932132,1.251793,-9.683860,2.710421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,-290.670074,107.864807,-10.905443,16.349510,-10.093550,18.151892,-25.673592,-28.731098,-13.618978,-15.610921,-22.825590,-5.592656,16.369370,-14.455715,6.453847,2.919372,-22.993980,-9.529423,1.558635,-4.659712
3426,-206.482544,135.750595,-35.974678,53.950138,-13.356145,23.576660,-3.269206,6.210017,-15.155455,-13.114673,-14.369177,-14.378550,8.763562,-21.915689,-1.584412,-3.689015,5.678556,0.104813,-2.471504,6.251332
3427,-267.670593,99.501114,-13.876461,36.051384,-1.516481,15.851993,-19.727144,-11.796036,-19.238400,-16.816988,-20.405027,-3.661840,2.577270,-18.873972,3.944394,-6.898897,-5.065143,-11.858092,-2.611360,-10.019358
3428,-308.496521,88.060783,-10.513425,53.359203,-1.133422,25.417496,0.162233,12.318471,-0.035208,-5.947904,-10.562906,-14.901034,0.723643,-12.418535,-0.250707,-8.382144,5.308206,2.212369,-5.269159,0.034238


In [60]:
y = np.append(y_0, y_1)
y

array([0., 0., 0., ..., 1., 1., 1.])

In [62]:
y = pd.DataFrame(y)

In [64]:
y

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3425,1.0
3426,1.0
3427,1.0
3428,1.0


In [76]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(20,)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(40, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax'),
])

In [77]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [70]:
dataset = tf.data.Dataset.from_tensor_slices((X.values, y.values))

In [84]:
shuffled_dataset = dataset.shuffle(len(X)).batch(1)

In [85]:
val_dataset = shuffled_dataset.take(1000) 
train_dataset = shuffled_dataset.skip(1000)

In [86]:
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
2430/2430 [==============================] - 6s 2ms/step - loss: 3.4340e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
2430/2430 [==============================] - 3s 1ms/step - loss: 6.5273e-09 - accuracy: 1.0000 - val_loss: 1.0729e-09 - val_accuracy: 1.0000
Epoch 3/10
2430/2430 [==============================] - 3s 1ms/step - loss: 4.9077e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
2430/2430 [==============================] - 4s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
2430/2430 [==============================] - 4s 2ms/step - loss: 1.4723e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
2430/2430 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
2430/2430 [==============================] - 3s 1ms/step - loss: 0.0000e+